In [1]:
%pip install --upgrade openai python-dotenv

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------- -------- 0.8/1.0 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 5.4 MB/s  0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.5.0
    Uninstalling openai-2.5.0:
      Successfully uninstalled openai-2.5.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, json, time, random
from typing import List, Dict, Any
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter API key: ')

MODEL = os.getenv('OPENAI_MODEL', 'gpt-3.5-turbo')
client = OpenAI()

print('Model:', MODEL, '| API key present:', 'OPENAI_API_KEY' in os.environ)

Model: gpt-3.5-turbo | API key present: True


In [6]:
CATEGORIES = ['Product', 'Feature', 'Process', 'Growth', 'Ops', 'Reasearch', 'Content', 'Other']

WEIGHT_USER_VALUE = 0.5
WEIGHT_FEASIBILITY = 0.3
WEIGHT_NOVELTY = 0.2

DEFAULT_TEMPERATURE = 0.7
DEFAULT_MAX_TOKENS = 1200

RESPONSE_FORMAT = {"type": "json_object"}

def normalize_score(x) -> int:
    try:
        return max(0, min(10, int(round(float(x)))))
    except Exception:
        return 0
    
def impact(user_value: int, feasibility: int, novelty: int) -> float:
    return round((WEIGHT_USER_VALUE * user_value + 
                  WEIGHT_FEASIBILITY * feasibility + 
                  WEIGHT_NOVELTY * novelty) * 10, 1)

def chat_json(messages: List[Dict[str, str]], 
              temperature: float = DEFAULT_TEMPERATURE, 
              max_tokens: int = DEFAULT_MAX_TOKENS) -> Dict[str, Any]:
    
    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        response_format=RESPONSE_FORMAT,
    )
    return json.loads(resp.choices[0].message.content)